library import

In [39]:
#import libraries
import pandas as pd
import matplotlib
import numpy as np
import seaborn as sns
import scipy.stats as stats
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
from scipy.stats import iqr
import statsmodels.stats as sm_stats
from sklearn import preprocessing
import statsmodels.stats.api as sms
import statsmodels.api as sm
from scipy.stats import ttest_ind

files load

In [40]:
#load files
session = pd.read_csv("session.csv", sep='\t')
product = pd.read_csv("product.csv", sep='\t')
user = pd.read_csv("user.csv", sep='\t')

user columns drop

In [41]:
user.drop(columns=["address", "residence"], inplace=True)

user



,job,birthdate,name,sex,username,registration,user_id,mail,race
0,"Nurse, children's",1965-08-08,Ludmila Šimková,F,vladimirablahova,"09/18/2020, 00:00:00",782,hajekadam@chello.cz,NaN
1,NaN,NaN,Fabrizia Beccaria,NaN,paloma33,14 May 2021,2158,silvioiacobucci@libero.it,NaN
2,NaN,NaN,João Alves,NaN,eduardoda-rosa,2017/06/23,747,beniciocampos@yahoo.com.br,NaN
3,"Exhibitions officer, museum/gallery",2003-02-24,Zdeňka Krejčová,F,jandovanela,2019/05/16,2204,jelinkovapetra@email.cz,NaN
4,NaN,NaN,Juan Cunha,NaN,lara09,2019/10/28,720,stella46@bol.com.br,NaN
...,...,...,...,...,...,...,...,...,...
2574,NaN,NaN,Hans-Jochen Hande,M,ruppertsilva,"02/07/2015, 00:00:00",938,kasimirgrein-groth@yahoo.de,NaN
2575,NaN,NaN,Mariana Berengario,NaN,amontessori,"06/20/2020, 00:00:00",779,cabriniannamaria@yahoo.com,NaN
2576,NaN,1908-08-20,Univ.Prof. Margret Trubin,NaN,mangoldrabea,"02/21/2018, 00:00:00",729,hans-ottoritter@gmx.de,NaN
2577,Chirurgiemechaniker,1975-08-12,Andrew Segebahn,NaN,bjopich,21 Feb 2019,549,pgerlach@hotmail.de,NaN


registration date formatting

In [42]:
user['registration'] = pd.to_datetime(user['registration'], errors='coerce')
average=user['registration'].mean()
user['registration'].fillna(average, inplace=True)
user['registration'] = user['registration'].dt.date
user['registration']

0       2020-09-18
1       2021-05-14
2       2017-06-23
3       2019-05-16
4       2019-10-28
           ...    
2574    2015-02-07
2575    2020-06-20
2576    2018-02-21
2577    2019-02-21
2578    2017-06-09
Name: registration, Length: 2579, dtype: object

name title deletion

In [43]:
def clean_name(name):
    if '.' in name:
        name_parts = name.split()
        name_parts = [part for part in name_parts if '.' not in part]
        return ' '.join(name_parts)
    else:
        return name

user['name'] = user['name'].apply(clean_name)

user["name"]

0          Ludmila Šimková
1        Fabrizia Beccaria
2               João Alves
3          Zdeňka Krejčová
4               Juan Cunha
               ...        
2574     Hans-Jochen Hande
2575    Mariana Berengario
2576        Margret Trubin
2577       Andrew Segebahn
2578       Gino Cociarelli
Name: name, Length: 2579, dtype: object

user email and id fixing

In [44]:
#delete duplicated email rows
print("Duplicated email count: ",user["mail"].duplicated().sum())
user.drop_duplicates(subset='mail', keep='first', inplace=True)
print("Duplicated email count after fix: ",user["mail"].duplicated().sum())

print("Duplicated user_id count: ",user["user_id"].duplicated().sum())

# Create a new column called 'old_user_id' to store the original user_id
user["old_user_id"] = user['user_id'].copy()

# create a list to store unique user_id
unique_id = []

for index, row in user.iterrows():
    user_id = row['user_id']
    
    # Check if user_id is unique
    if user_id not in unique_id:
        unique_id.append(user_id)
    else:
        # Assign a new unique user_id
        new_user_id = max(unique_id) + 1
        unique_id.append(new_user_id)
        user.at[index, 'user_id'] = new_user_id

print("Duplicated user_id count after fix:",user["user_id"].duplicated().sum())

Duplicated email count:  6
Duplicated email count after fix:  0
Duplicated user_id count:  948
Duplicated user_id count after fix: 0


session screen width and heigth fixing

In [45]:
valid_lower = [[640, 360], [640, 480], [800, 600], [1024, 768], [1280, 720], [1360, 768], [1366, 768], [1920, 1080]]
valid_higher = [[2560, 600], [2048, 1152], [2048, 1152], [2560, 1080], [2560, 1440], [2560, 1600], [3440, 1440], [3840, 2160]]

# Create new columns with int64 data type
session['new_screen_width'] = 0
session['new_screen_height'] = 0

for i in range(len(session)):
    if session["screen_height"].iloc[i] == "lower" and session["screen_width"].iloc[i] == "lower":
        session.at[i, "new_screen_height"] = 640
        session.at[i, "new_screen_width"] = 360
    elif session["screen_height"].iloc[i] == "higher" and session["screen_width"].iloc[i] == "higher":
        session.at[i, "new_screen_height"] = 2048
        session.at[i, "new_screen_width"] = 1152
    elif session["screen_height"].iloc[i] == "lower":
        for item in valid_lower:
            if session["screen_width"].iloc[i] == str(item[0]):
                session.at[i, "new_screen_height"] = item[1]
                session.at[i, "new_screen_width"] = item[0]
    elif session["screen_width"].iloc[i] == "lower":
        for item in valid_lower:
            if session["screen_height"].iloc[i] == str(item[1]):
                session.at[i, "new_screen_width"] = item[0]
                session.at[i, "new_screen_height"] = item[1]
    elif session["screen_height"].iloc[i] == "higher":
        for item in valid_higher:
            if session["screen_width"].iloc[i] == str(item[0]):
                session.at[i, "new_screen_height"] = item[1]
                session.at[i, "new_screen_width"] = item[0]
    elif session["screen_width"].iloc[i] == "higher":
        for item in valid_higher:
            if session["screen_height"].iloc[i] == str(item[1]):
                session.at[i, "new_screen_width"] = item[0]
                session.at[i, "new_screen_height"] = item[1]
    else:
        session.at[i, "new_screen_width"] = int(session["screen_width"].iloc[i])
        session.at[i, "new_screen_height"] = int(session["screen_height"].iloc[i])


session.drop(columns=['screen_width', 'screen_height'], inplace=True)

session.rename(columns={'new_screen_width': 'screen_width', 'new_screen_height': 'screen_height'}, inplace=True)


print("screen_height data type:",session["screen_height"].dtypes)
print("screen_widthdata type:",session["screen_width"].dtypes)


screen_height data type: int64
screen_widthdata type: int64


session duplicated rows drop

In [46]:
print("Duplicated rows:",session.duplicated().sum())
session.drop_duplicates( keep='first', inplace=True)

print("Duplicated rows after fix:",session.duplicated().sum())
print("Duplicated session_id count: ",session["session_id"].duplicated().sum())

unique_id = []

for index, row in session.iterrows():
    session_id = row['session_id']
    
    # Check if session_id is unique
    if session_id not in unique_id:
        unique_id.append(session_id)
    else:
        # Assign a new unique user_id
        new_session_id = max(unique_id) + 1
        unique_id.append(new_session_id)
        session.at[index, 'session_id'] = new_session_id

print("Duplicated session_id count after fix:",session["session_id"].duplicated().sum())


Duplicated rows: 131
Duplicated rows after fix: 0
Duplicated session_id count:  42
Duplicated session_id count after fix: 0


In [47]:
user.drop(columns=["job", "name", "username", "mail"], inplace=True)
user

,birthdate,sex,registration,user_id,race,old_user_id
0,1965-08-08,F,2020-09-18,782,NaN,782
1,NaN,NaN,2021-05-14,2158,NaN,2158
2,NaN,NaN,2017-06-23,747,NaN,747
3,2003-02-24,F,2019-05-16,2204,NaN,2204
4,NaN,NaN,2019-10-28,720,NaN,720
...,...,...,...,...,...,...
2574,NaN,M,2015-02-07,938,NaN,938
2575,NaN,NaN,2020-06-20,779,NaN,779
2576,1908-08-20,NaN,2018-02-21,729,NaN,729
2577,1975-08-12,NaN,2019-02-21,3521,NaN,549


data export

In [48]:
product.to_csv('product_phase_1.csv', sep='\t', index=False)
user.to_csv('user_phase_1.csv', sep='\t', index=False)
session.to_csv('session_phase_1.csv', sep='\t', index=False)
